In [ ]:
"""
First, we list all files copied into the Unity Catalog Volume using dbutils. The returned list is actually a list of objects which have certain properties such as "name" and "path" which can be returned using (.)notation such as object.name
"""
# Storage account information
storage_account = "ubsadatabrickspoc2"
storage_container = "databricks-poc"

# Get list of files from Volumes
source_files = dbutils.fs.ls("/Volumes/umpqua_poc_dev/bronze_data/bronze_volume/sample_data/csv_files")

# For each file object in list of file objects
for source_file in source_files:
    """
    This section is used to parse & clean the input directory objects, and to create the three destination paths. 
    """
    # Path to the csv file
    csv_path = source_file.path
    print(f"Processing CSV File: {csv_path}")
    
    # Name of the csv file slightly cleaned (all lowercase, " " replaced with "_", and any "." removed)
    name = "_".join(source_file.name.split(".")[:-1]).replace(" ", "_").lower()
    print(f"Cleaned file name: {name}")

    # Path to Volumes where "||" delim file will be saved
    pipe_delim_path = f"/Volumes/umpqua_poc_dev/bronze_data/bronze_volume/sample_data/pipe_delim_files/{name}"
    print(f"Volume Destination Path for || delim file: {pipe_delim_path}")

    # Path to landing zone in ADLS where "||" delim file will be saved
    auto_loader_path = f"abfss://{storage_container}@{storage_account}.dfs.core.windows.net/umpqua_poc/landing_zone/{name}"
    print(f"Landing Zone Destination Path for || delim file: {auto_loader_path}")

    # Unity Catalog path to create test table
    table_path = f"umpqua_poc_dev.bronze_data.test_{name}"
    print(f"Unity Catalog Test Table Path: {table_path}")


    """
    Here, we will read the "," delimited csv file; write it to a Unity Catalog Volume as a "||" delimited csv file; write it to our simulated landing zone as a "||" delimited file (AutoLoader will be used to process files from this location); and finally we will create a Unity Catalog Delta Table. These are redundant and largely unecessary steps that are being done simply to illustrate the process. 
    """
    # Read the csv file with "," delimiter; with first row as header; without inferring the schema
    print(f"Reading CSV File: {csv_path}")
    df = spark.read.format("csv").option("delimiter", ",").option("header", "True").load(csv_path)
    print(f"Successfully Read CSV File: {csv_path}")

    # Write csv file as "||" delim to Volumes for easy access in the future; Including the header; Overwriting any existing data
    print(f"Saving to Volume: {pipe_delim_path}")
    df.write.csv(path=pipe_delim_path, sep="||", header=True, mode="overwrite")
    print(f"Successfully Wrote to Volume: {pipe_delim_path}")

    # Write csv file as "||" delim to ADLS landing zone; Including the header; Overwriting any existing data; 
    # This will be the AutoLoader target
    print(f"Saving to landing zone: {auto_loader_path}")
    df.write.csv(path=auto_loader_path,sep="||",header=True,mode="overwrite")
    print(f"Successfully Wrote to Landing Zone: {auto_loader_path}")

    # Save as Delta table in Unity Catalog
    print(f"Creating test table: {table_path}")
    df.write.format("delta").mode("overwrite").saveAsTable(table_path)
    print(f"Successfully Saved to Unity Catalog Delta Table: {table_path}")

Processing CSV File: dbfs:/Volumes/umpqua_poc_dev/bronze_data/bronze_volume/sample_data/csv_files/Account Notification.csv
Cleaned file name: account_notification
Volume Destination Path for || delim file: /Volumes/umpqua_poc_dev/bronze_data/bronze_volume/sample_data/pipe_delim_files/account_notification
Landing Zone Destination Path for || delim file: abfss://databricks-poc@oneenvadls.dfs.core.windows.net/umpqua_poc/landing_zone/account_notification
Unity Catalog Test Table Path: umpqua_poc_dev.bronze_data.test_account_notification
Reading CSV File: dbfs:/Volumes/umpqua_poc_dev/bronze_data/bronze_volume/sample_data/csv_files/Account Notification.csv
Successfully Read CSV File: dbfs:/Volumes/umpqua_poc_dev/bronze_data/bronze_volume/sample_data/csv_files/Account Notification.csv
Saving to Volume: /Volumes/umpqua_poc_dev/bronze_data/bronze_volume/sample_data/pipe_delim_files/account_notification
Successfully Wrote to Volume: /Volumes/umpqua_poc_dev/bronze_data/bronze_volume/sample_data/p